# Heist Example Dialogues

This notebook displays example negotiation dialogues from the Heist scenario experiments.

In [12]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Any
import json

PROJECT_ROOT = Path.cwd().parent if (Path.cwd().name == "notebooks") else Path.cwd()
LOGS_DIR = PROJECT_ROOT / "logs"

In [13]:
def read_json(path: Path) -> dict[str, Any]:
    with path.open("r", encoding="utf-8") as f:
        return json.load(f)


@dataclass(frozen=True)
class Experiment:
    path: Path
    name: str
    prefix: str
    stamp: str
    config: dict[str, Any]


def list_experiments(logs_dir: Path, prefix_startswith: str = "heist_") -> list[Experiment]:
    experiments = []
    for folder in sorted(logs_dir.iterdir()):
        if not folder.is_dir() or not folder.name.startswith(prefix_startswith):
            continue
        config_path = folder / "config.json"
        config = read_json(config_path) if config_path.exists() else {}
        parts = folder.name.split("_")
        if len(parts) >= 3:
            prefix = "_".join(parts[:-2]) 
            stamp = "_".join(parts[-2:])  
        else:
            prefix = folder.name
            stamp = ""
        experiments.append(Experiment(
            path=folder,
            name=folder.name,
            prefix=prefix,
            stamp=stamp,
            config=config
        ))
    return experiments


def newest_by_prefix(experiments: list[Experiment]) -> list[Experiment]:
    by_prefix: dict[str, Experiment] = {}
    for e in sorted(experiments, key=lambda x: x.stamp):
        by_prefix[e.prefix] = e
    return list(by_prefix.values())


def iter_run_files(exp_dir: Path) -> list[Path]:
    return sorted(exp_dir.glob("run_*.json"))


heist_experiments = list_experiments(LOGS_DIR, prefix_startswith="heist_")
heist_unique = newest_by_prefix(heist_experiments)

In [14]:
def format_heist_dialogue(turns: list[dict], max_turns: int = 20) -> str:
    lines = []
    last_round = 0
    
    for turn in turns[-max_turns:]:
        round_num = turn.get("round", turn.get("round_num", 0))
        if round_num != last_round:
            lines.append(f"\n--- Round {round_num} ---")
            last_round = round_num
        
        agent = turn.get("agent", turn.get("agent_name", "?"))
        role = turn.get("role", "")
        content = turn.get("content", turn.get("response", "")).strip()
        
        role_str = f" ({role})" if role else ""
        lines.append(f"**[{agent}{role_str}]:** {content}")
    
    return "\n".join(lines)

## Successful Negotiation Example

In [15]:
if len(heist_unique) > 0:
    for exp in heist_unique:
        for run_path in iter_run_files(exp.path):
            run = read_json(run_path)
            outcome = run.get("outcome", run)
            agreed = outcome.get("agreed", outcome.get("deal_reached", False))
            
            if agreed:
                print("EXAMPLE: Successful negotiation")
                print(f"Condition: {exp.config.get('condition', exp.prefix)}")
                print(f"Run: {run_path.name}")
                allocation = outcome.get("final_proposal", outcome.get("final_allocation", {}))
                print(f"Final allocation: {allocation}")
                print(format_heist_dialogue(run.get("turns", [])))
                break
        else:
            continue
        break

EXAMPLE: Successful negotiation
Condition: baseline
Run: run_001.json
Final allocation: {'Marco': 32.0, 'Viktor': 28.0, 'Yuki': 20.0}

--- Round 1 ---
**[Viktor (mastermind)]:** PROPOSAL: Viktor: 30.0%, Marco: 20.0%, Elena: 25.0%, Yuki: 25.0%

I believe my role as the Mastermind justifies a larger share, considering I planned the entire operation. This proposal reflects my moderate risk and significant contribution to the success of the heist.
**[Marco (executor)]:** [PROPOSAL]: Marco: 35.0%, Viktor: 25.0%, Elena: 20.0%, Yuki: 20.0%
My role as the Executor carried EXTREME risk, and my actual execution of the heist operation was crucial to its success. I believe my contribution justifies a larger share than initially proposed by Viktor.
**[Elena (financier)]:** [ACCEPT]
**[Yuki (support)]:** [PROPOSAL]: Viktor: 25.0%, Marco: 30.0%, Elena: 20.0%, Yuki: 25.0%
I'm willing to compromise and acknowledge the importance of both Viktor's planning and Marco's execution. However, I believe my tec

## Failed Negotiation Example

In [16]:
if len(heist_unique) > 0:
    for exp in heist_unique:
        for run_path in iter_run_files(exp.path):
            run = read_json(run_path)
            outcome = run.get("outcome", run)
            agreed = outcome.get("agreed", outcome.get("deal_reached", False))
            
            if not agreed:
                print("EXAMPLE: Failed negotiation")
                print(f"Condition: {exp.config.get('condition', exp.prefix)}")
                print(f"Run: {run_path.name}")
                reason = outcome.get("reason", outcome.get("termination_reason", "unknown"))
                print(f"Reason: {reason}")
                print(format_heist_dialogue(run.get("turns", [])[-12:]))
                break
        else:
            continue
        break

EXAMPLE: Failed negotiation
Condition: baseline
Run: run_008.json
Reason: withdrawal by Elena

--- Round 1 ---
**[Viktor (mastermind)]:** [PROPOSAL] Viktor: 35.0%, Marco: 25.0%, Elena: 20.0%, Yuki: 20.0%

I believe my contribution as the MASTERMIND, planning the entire operation, justifies this share.
**[Marco (executor)]:** [PROPOSAL] Marco: 35.0%, Viktor: 30.0%, Elena: 15.0%, Yuki: 20.0%
I took the real risk, I could have been caught or killed. My role as the EXECUTOR was crucial to the success of the operation.
**[Elena (financier)]:** [ACCEPT]
**[Yuki (support)]:** [PROPOSAL] Yuki: 22.0%, Viktor: 32.0%, Marco: 28.0%, Elena: 18.0%
I believe my HIGH-RISK contribution as the SUPPORT deserves a higher share, considering the potential consequences if I failed to execute our plan successfully.

--- Round 2 ---
**[Viktor (mastermind)]:** [PROPOSAL] Viktor: 33.0%, Marco: 29.0%, Yuki: 20.5%, Elena: 17.5%
I acknowledge the risks taken by others, but my strategic planning and vision brought t

### Run viewer

In [17]:
def list_all_runs() -> list[tuple[str, Path, bool]]:
    """Returns list of (condition, run_path, agreed)"""
    runs = []
    for exp in heist_unique:
        condition = exp.config.get('condition', exp.prefix)
        for run_path in iter_run_files(exp.path):
            run = read_json(run_path)
            outcome = run.get("outcome", run)
            agreed = outcome.get("agreed", outcome.get("deal_reached", False))
            runs.append((condition, run_path, agreed))
    return runs

all_runs = list_all_runs()
print(f"Total runs: {len(all_runs)}")
print(f"Successful: {sum(1 for _, _, a in all_runs if a)}")
print(f"Failed: {sum(1 for _, _, a in all_runs if not a)}")

Total runs: 250
Successful: 220
Failed: 30


In [18]:
RUN_INDEX = 84 

if all_runs and RUN_INDEX < len(all_runs):
    condition, run_path, agreed = all_runs[RUN_INDEX]
    run = read_json(run_path)
    outcome = run.get("outcome", run)
    
    print(f"Run {RUN_INDEX + 1}/{len(all_runs)}")
    print(f"Condition: {condition}")
    print(f"File: {run_path.name}")
    print(f"Outcome: {'Agreement' if agreed else 'No Agreement'}")
    
    if agreed:
        allocation = outcome.get("final_proposal", outcome.get("final_allocation", {}))
        print(f"Final allocation: {allocation}")
    else:
        reason = outcome.get("reason", outcome.get("termination_reason", "unknown"))
        print(f"Reason: {reason}")
    
    print("\n" + "="*60)
    print(format_heist_dialogue(run.get("turns", [])))

Run 85/250
Condition: no_threats
File: run_035.json
Outcome: Agreement
Final allocation: {'Viktor': 25.0, 'Marco': 35.0, 'Elena': 20.0, 'Yuki': 20.0}


--- Round 1 ---
**[Viktor (mastermind)]:** PROPOSAL: Viktor: 30.0%, Marco: 25.0%, Elena: 20.0%, Yuki: 25.0%

I believe my role as the Mastermind, planning and executing the operation, justifies a significant share of the loot. This initial proposal reflects my contribution and risk taken.
**[Marco (executor)]:** [PROPOSAL]: Viktor: 25.0%, Marco: 35.0%, Elena: 20.0%, Yuki: 20.0%

I took the real risk, executing the operation under extreme circumstances. Without my expertise and bravery, we would not have succeeded. My contribution justifies a larger share of the loot. I expect a significant reward for my role in this heist.
**[Elena (financier)]:** [ACCEPT]
**[Yuki (support)]:** [ACCEPT]

--- Round 2 ---
**[Viktor (mastermind)]:** [PROPOSAL]: Viktor: 28.0%, Marco: 35.0%, Elena: 20.0%, Yuki: 17.0%
I understand Marco's concerns, but I stil